In [37]:
from pymongo import MongoClient
import pymongo
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pickle
from tmqr.settings import *
from tmqrfeed.contracts import OptionContract

In [2]:
instrument = 'TST'
market = 'T'
future = 'F-TST-H11-110322'
opt_expiration = '110322'

In [3]:
options = ['C', 'P']
N = 60

In [4]:
strikes = list(np.arange(50.0, 100.0, 1.0))

In [5]:
options_tickers_list = []
for opt_type in options:
    for s in strikes:
        #US.C.F-CL-H11-110322.110121@89.0
        options_tickers_list.append("{0}.{1}.{2}.{3}@{4}".format(market, opt_type, future,  opt_expiration, s))        

In [6]:
dates = [datetime(2011, 1, 2) + timedelta(days=x) for x in range(N)]

client = MongoClient(MONGO_CONNSTR)
db = client[MONGO_DB]

In [7]:


for i, opt in enumerate(options_tickers_list):
    iv_arr = np.random.random(size=N)
    price_arr = np.random.random(size=N)
    to_exp = np.arange(N)
    
    df = pd.DataFrame({'iv': iv_arr, 'px': price_arr, 'toexp': to_exp}, index=dates)
    #
    # Writing bundled quotes
    # 
    db['quotes_options_eod_bundled'].create_index([('tckr', pymongo.ASCENDING)])
    db['quotes_options_eod_bundled'].create_index([('idx', pymongo.ASCENDING)])
    db['quotes_options_eod_bundled'].replace_one({'tckr': opt},
                                                 {'tckr': opt, 
                                                  'data': pickle.dumps(df),
                                                  'idx': i,
                                                 },
                                                 upsert=True
    )
    #
    # Writing single records
    # 
    db['quotes_options_eod_single'].create_index([('tckr', pymongo.ASCENDING), ('dt', pymongo.ASCENDING)])
    db['quotes_options_eod_single'].create_index([('idx', pymongo.ASCENDING)])
    for i, dt in enumerate(dates):
        """print({
            'tckr': opt,
            'dt': dt,
            'iv': iv_arr[i],
            'px': price_arr[i],
            'to_exp': to_exp[i]
        })
        """
        db['quotes_options_eod_single'].replace_one({'tckr': opt, 'dt': dt},
            {
                'tckr': opt,
                'dt': dt,
                'iv': iv_arr[i],
                'px': price_arr[i],
                'idx': i,
                #'to_exp': to_exp[i]
            }, upsert=True)
    
    
    
    

In [8]:
class ChainSingle:
    def __init__(self, opt_tickers):
        client = MongoClient(MONGO_CONNSTR)
        self.db = client[MONGO_DB]
        self.tickers = opt_tickers
        
    def compose(self, date):
        result = db['quotes_options_eod_single'].find({'tckr': {'$in': self.tickers}, 'dt': date})
        #result = db['quotes_options_eod_single'].find({'idx': {'$in': list(range(len(self.tickers)))}, 'dt': date})
        return pd.DataFrame(list(result))
    
    def get_one(self, tckr, date):
        result = db['quotes_options_eod_single'].find({'tckr': tckr, 'dt': date})
        return pd.DataFrame(list(result))
        
class ChainBundled:
    def __init__(self, opt_tickers):
        client = MongoClient(MONGO_CONNSTR)
        self.db = client[MONGO_DB]
        self.tickers = opt_tickers
        self.panel = None
        self.one_df = None
        
    def compose(self, date):
        if self.panel is None:
            result = db['quotes_options_eod_bundled'].find({'tckr': {'$in': self.tickers}})  
            #result = db['quotes_options_eod_bundled'].find({'idx': {'$in': list(range(len(self.tickers)))}})  
            
            res_dict = {}
            for res in result:
                res_dict[res['tckr']] = pickle.loads(res['data'])

            self.panel = pd.Panel(res_dict)
        return self.panel.major_xs(date)
    
    def get_one(self, tckr, date):
        if self.one_df is None:
            result = db['quotes_options_eod_bundled'].find_one({'tckr': tckr})        
        
            self.one_df = pickle.loads(result['data'])
        return self.one_df.loc[date]
        
        
        

# Chaing composition benchmark

In [13]:
#%%timeit 
chain1 = ChainSingle(options_tickers_list)
chain1.compose(datetime(2011, 1, 2))

,_id,dt,idx,iv,px,tckr
0,58d0ea006db8b05216969902,2011-01-02,0,0.090951,0.871217,T.C.F-TST-H11-110322.110322@50.0
1,58d0ea006db8b0521696993e,2011-01-02,0,0.263234,0.321990,T.C.F-TST-H11-110322.110322@51.0
2,58d0ea006db8b0521696997a,2011-01-02,0,0.714948,0.477305,T.C.F-TST-H11-110322.110322@52.0
3,58d0ea006db8b052169699b6,2011-01-02,0,0.770159,0.047257,T.C.F-TST-H11-110322.110322@53.0
4,58d0ea006db8b052169699f2,2011-01-02,0,0.328547,0.616885,T.C.F-TST-H11-110322.110322@54.0
5,58d0ea006db8b05216969a2e,2011-01-02,0,0.891535,0.825014,T.C.F-TST-H11-110322.110322@55.0
6,58d0ea006db8b05216969a6a,2011-01-02,0,0.795934,0.765882,T.C.F-TST-H11-110322.110322@56.0
7,58d0ea006db8b05216969aa6,2011-01-02,0,0.068347,0.893364,T.C.F-TST-H11-110322.110322@57.0
8,58d0ea006db8b05216969ae2,2011-01-02,0,0.574862,0.462584,T.C.F-TST-H11-110322.110322@58.0
9,58d0ea006db8b05216969b1e,2011-01-02,0,0.288212,0.694716,T.C.F-TST-H11-110322.110322@59.0


In [14]:
#%%timeit
chain2 = ChainBundled(options_tickers_list)
chain2.compose(datetime(2011, 1, 2))

,T.C.F-TST-H11-110322.110322@50.0,T.C.F-TST-H11-110322.110322@51.0,T.C.F-TST-H11-110322.110322@52.0,T.C.F-TST-H11-110322.110322@53.0,T.C.F-TST-H11-110322.110322@54.0,T.C.F-TST-H11-110322.110322@55.0,T.C.F-TST-H11-110322.110322@56.0,T.C.F-TST-H11-110322.110322@57.0,T.C.F-TST-H11-110322.110322@58.0,T.C.F-TST-H11-110322.110322@59.0,...,T.P.F-TST-H11-110322.110322@90.0,T.P.F-TST-H11-110322.110322@91.0,T.P.F-TST-H11-110322.110322@92.0,T.P.F-TST-H11-110322.110322@93.0,T.P.F-TST-H11-110322.110322@94.0,T.P.F-TST-H11-110322.110322@95.0,T.P.F-TST-H11-110322.110322@96.0,T.P.F-TST-H11-110322.110322@97.0,T.P.F-TST-H11-110322.110322@98.0,T.P.F-TST-H11-110322.110322@99.0
iv,0.090951,0.263234,0.714948,0.770159,0.328547,0.891535,0.795934,0.068347,0.574862,0.288212,...,0.942456,0.218043,0.908934,0.068523,0.971011,0.234693,0.436615,0.687888,0.025676,0.903587
px,0.871217,0.321990,0.477305,0.047257,0.616885,0.825014,0.765882,0.893364,0.462584,0.694716,...,0.751029,0.035038,0.315864,0.919912,0.368776,0.036495,0.289341,0.242061,0.077101,0.464137
toexp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [15]:
p = chain2.panel

In [33]:
mem = 0
for i in p:
    mem += p.loc[i].memory_usage()
print(mem)


Index    48000
iv       48000
px       48000
toexp    48000
dtype: int64


In [26]:
%%timeit
p.loc['T.C.F-TST-H11-110322.110322@50.0'].at[datetime(2011, 1, 2), 'iv']

The slowest run took 6.77 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 52.7 µs per loop


In [11]:
%timeit chain1.compose()

NameError: name 'chain1' is not defined

In [12]:
%timeit chain2.compose(datetime(2011, 1, 3))

NameError: name 'chain2' is not defined

In [ ]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
for i in range(20):
    chain1.compose(datetime(2011, 1, 2) + timedelta(days=i))

In [ ]:
%%timeit 
chain2 = ChainBundled(options_tickers_list)

for i in range(20):
    chain2.compose(datetime(2011, 1, 2) + timedelta(days=i))

# Single element fetching benchmark

In [ ]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
chain1.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2))

In [ ]:
%%timeit 
chain2 = ChainBundled(options_tickers_list)
chain2.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2))

In [ ]:
%%timeit 
chain1 = ChainSingle(options_tickers_list)
for i in range(20):
    chain1.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2) + timedelta(days=i))

In [ ]:
%%timeit 
chain2 = ChainBundled(options_tickers_list)

for i in range(20):
    chain2.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2) + timedelta(days=i))

In [ ]:
chain2 = ChainBundled(options_tickers_list)
chain2.get_one('T.C.F-TST-H11-110322.110322@50.0', datetime(2011, 1, 2))

# Option chains benchmarks

In [72]:
#%%timeit 
cursor = db['asset_index'].aggregate([
            {'$match': {
                'underlying': 'US.F.ES.H11.110318',
                'type': {'$in': ['P', 'C']},                
            }},
            
            {'$sort': {'strike': 1}},
            
            {'$project': {'tckr': 1, 'exp': 1, 'strike': 1, 'type': 1}
            },
            
            {'$group': {
                '_id': {'date': '$exp'},
                'chain': {'$push': '$$ROOT'},
            }
            },
            {'$sort': {"_id.date": 1}}          
])
chain_list = list(cursor)

In [54]:
%timeit [OptionContract(opt['tckr']) for opt in chain_list[0]['chain']]

100 loops, best of 3: 9.89 ms per loop


In [71]:
len(chain_list)

3

In [74]:
#%%timeit 
from collections import OrderedDict

chain_result = OrderedDict()

strike_count = 0

for exp in chain_list:
    options = chain_result.setdefault(exp['_id']['date'], OrderedDict())
    
    prev_strike = 0.0
    chain = exp['chain']
    for i, strike_rec in enumerate(chain):
        strike_count += 1
        strike = strike_rec['strike']
        if i == 0:
            continue
                
        if strike == chain[i-1]['strike']:
            # We have put call pair             
            if strike_rec['type'] == 'C':
                call_idx = i
                put_idx = i-1
            else:
                call_idx = i-1
                put_idx = i
            
            options[strike] = (chain[call_idx]['tckr'], chain[put_idx]['tckr'])    
    

In [75]:
strike_count

1110

In [73]:
with open("chain_list_es.pkl", 'wb') as f:
    pickle.dump(chain_list, f)

In [146]:
[x['_id']['date'] for x in chain_list]

[datetime.datetime(2011, 1, 21, 0, 0),
 datetime.datetime(2011, 2, 18, 0, 0),
 datetime.datetime(2011, 3, 18, 0, 0)]